In [1]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec.load_word2vec_format('Base/GoogleNews-vectors-negative300.bin', binary=True)

#how don't archive in memory

In [ ]:
model.most_similar(positive=['woman', 'king'], negative=['man'], topn=7)

In [2]:
import numpy as np

with open('Base/food_words.txt', 'r') as infile:
    food_words = infile.readlines()
    
with open('Base/sports_words.txt', 'r') as infile:
    sport_words = infile.readlines()
    
with open('Base/weather_words.txt', 'r') as infile:
    weather_words = infile.readlines()
    

def getWordVecs(words):
    vecs = []
    for word in words:
        word = word.replace('\n', '')
        try:
            vecs.append(model[word])
        except KeyError:
            continue
    #vecs = np.concatenate(vecs)         #how do you mean concatenate vectors
    return vecs
    #return np.array(vecs, dtype='float') #TSNE expects float type values

food_vecs = getWordVecs(food_words)
sports_vecs = getWordVecs(sport_words)
weather_vecs = getWordVecs(weather_words)

In [3]:
len(food_vecs)

1

In [ ]:
from scipy.spatial.distance import cdist

# vecs: an array of real vectors
def crp(vecs):
    clusterVec = [0]         # tracks sum of vectors in a cluster
    clusterIdx = []         # array of index arrays. e.g. [[1, 3, 5], [2, 4, 6]]
    ncluster = 0
    # probablity to create a new table if new customer
    # is not strongly "similar" to any existing table
    pnew = 1.0/ (1 + ncluster)  
    N = len(vecs)
    rands = np.random.rand(N)         # N rand variables sampled from U(0, 1)
 
    for i in range(N):
        maxSim = 0
        maxIdx = 0
        v = vecs[i]
        for j in range(ncluster):
            if len(clusterVec) == 0:
                continue
            else:
                sim = cdist(clusterVec[j], v, 'cosine')
            if sim < maxSim:
                maxIdx = j
                maxSim = sim
            if maxSim < pnew:
                if rands(i) < pnew:
                    clusterVec[ncluster] = v
                    clusterIdx[ncluster] = [i]
                    ncluster += 1
                    pnew = 1.0 / (1 + ncluster)
                continue
        clusterVec[maxIdx] = clusterVec[maxIdx] + v
        print maxIdx
        clusterIdx.insert(maxIdx, i)
 
    return clusterIdx

In [ ]:
from scipy.spatial.distance import cdist

clusterVec = []         # tracks sum of vectors in a cluster
clusterIdx = []         # array of index arrays. e.g. [[1, 3, 5], [2, 4, 6]]
ncluster = 0
# probablity to create a new table if new customer
# is not strongly "similar" to any existing table
pnew = 1.0/ (1 + ncluster)  
N = len(food_vecs[0])
rands = np.random.rand(N)         # N rand variables sampled from U(0, 1)
 
for i in range(N):
    maxSim = 0
    maxIdx = 0
    #t = np.array([np.mean(food_vecs,axis=0)])
    v = food_vecs[i]
    for j in range(ncluster):
        print v
        if len(clusterVec) != 0:
            sim = cdist(clusterVec[j], v, 'cosine')
            print sim
            if sim < maxSim:
                maxIdx = j
                maxSim = sim
                if maxSim < pnew:
                    if rands(i) < pnew:
                        clusterVec[ncluster] = v
                        clusterIdx[ncluster] = [i]
                        ncluster += 1
                        pnew = 1.0 / (1 + ncluster)
                continue
        elif len(clusterVec) == 0:
                #clusterVec = v
                clusterVec = np.array([np.mean(v,axis=0)])
                clusterIdx.insert(maxIdx, i)
        else:
            clusterVec[maxIdx] = clusterVec[maxIdx] + v
            clusterVec = np.array([np.mean(clusterVec,axis=0)])
            #print maxIdx
            clusterIdx.insert(maxIdx, i)

In [ ]:
clusterIdx

In [ ]:
vec = []
vec.append(0.120000)
vec.append(0.190000)
vec.append(2.220000)
teste = crp(food_vecs[0])
#teste

In [ ]:
vetor = []
vetor.append(1)
vetor.append(2)
#vetor[0] = 3
vetor

In [ ]:
#vetor = np.cumsum(food_vecs) #how does cumsum make?
#either I sum all of them or I sum them apllying PCA

m = np.array([np.mean(food_vecs,axis=0), np.mean(sports_vecs,axis=0), np.mean(weather_vecs,axis=0)])

#print food_vecs.shape
print np.mean(m,axis=0).shape
#import gensim
#print gensim.__version__

In [ ]:
from scipy.spatial.distance import cdist

#idx = np.argmin(cdist(m,[model['acorn']],'cosine'),axis=0)
cdist(m,[model['eat']],'cosine')



In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
vetor_res = pca.fit_transform(food_vecs)

In [ ]:
vetor_res[0]

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

ts = TSNE(2)
reduced_vecs = ts.fit_transform(np.concatenate((food_vecs, sports_vecs, weather_vecs)))

#color points by word group to see if Word2Vec can separate them
for i in range(len(reduced_vecs)):
    if i < len(food_vecs):
        #food_words colored blue
        color = 'b'
    elif i >= len(food_vecs) and i < (len(food_vecs) + len(sports_vecs)):
        #sports_words colored red
        color = 'r'
    else:
        #weather_words colored green
        color = 'g'
    plt.plot(reduced_vecs[i,0], reduced_vecs[i,1], marker = 'o', color = color, markersize = 8)
    

In [ ]:
plt.show()